# EDA

## Libraries

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Code

In [65]:
airport_df=pd.read_csv(r'C:\Users\ekait\Downloads\airport_list_2.csv')
airport_df

,Location,Latitude,Longitude
0,ABE,40.652100,-75.440804
1,ABI,32.411301,-99.681900
2,ABQ,35.040199,-106.609001
3,ABR,45.449100,-98.421799
4,ABY,31.535500,-84.194504
...,...,...,...
383,XWA,48.257800,-103.741700
384,YAK,59.503300,-139.660004
385,YKM,46.568199,-120.543999
386,YNG,41.260700,-80.679100


In [68]:
df_station_latlon=pd.read_csv(r'C:\Users\ekait\Downloads\ghcnd-stations.csv')
weather_df = df_station_latlon[['ID', 'Latitude', 'Longitude']]
weather_df

C:\Users\ekait\AppData\Local\Temp\ipykernel_17828\2128011621.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_station_latlon=pd.read_csv(r'C:\Users\ekait\Downloads\ghcnd-stations.csv')


,ID,Latitude,Longitude
0,ACW00011604,17.1167,-61.7833
1,ACW00011647,17.1333,-61.7833
2,AE000041196,25.3330,55.5170
3,AEM00041194,25.2550,55.3640
4,AEM00041217,24.4330,54.6510
...,...,...,...
128019,ZI000067969,-21.0500,29.3670
128020,ZI000067975,-20.0670,30.8670
128021,ZI000067977,-21.0170,31.5830
128022,ZI000067983,-20.2000,32.6160


In [ ]:
from geopy.distance import geodesic

# Example: Load airport and weather station data
# Replace with your actual file paths
# airport_df = pd.read_csv("airports.csv")  # Should contain 'airport_name', 'latitude', 'longitude'
# weather_df = pd.read_csv("weather_stations.csv")  # Should contain 'station_name', 'latitude', 'longitude'

# Function to compute the geodesic distance (haversine distance)
def compute_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).km

# Find the closest weather station for each airport
closest_stations = []
for _, airport in airport_df.iterrows():
    airport_coords = (airport["Latitude"], airport["Longitude"])
    
    # Compute distance to each weather station
    weather_df["Distance"] = weather_df.apply(
        lambda station: compute_distance(airport_coords[0], airport_coords[1], 
                                         station["Latitude"], station["Longitude"]), axis=1
    )
    
    # Find the closest weather station
    nearest_station = weather_df.loc[weather_df["Distance"].idxmin()]
    
    # Store the result
    closest_stations.append({
        "Airport_Code": airport["Location"],
        "Airport_Lat": airport["Latitude"],
        "Airport_Lon": airport["Longitude"],
        "Closest_Weather_Station": nearest_station["ID"],
        "Station_Lat": nearest_station["Latitude"],
        "Station_Lon": nearest_station["Longitude"],
        "Distance_km": nearest_station["Distance"]
    })

# Convert to DataFrame
closest_stations_df = pd.DataFrame(closest_stations)

# Save results
closest_stations_df.to_csv(r"C:\Users\ekait\Downloads\airport_with_nearest_weather_station.csv", index=False)

## Done

In [3]:
# Read parquet file
raw = pd.read_parquet(r'C:\Users\ekait\Git\DSC288R_Capstone\data\processed\combined_data.parquet')

In [4]:
# Check head
raw.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2018-01-23,Endeavor Air Inc.,ABY,ATL,False,False,1202,1157.0,0.0,-5.0,...,1211.0,1249.0,7.0,1304,-8.0,0.0,-1.0,1300-1359,1,0.0
1,2018-01-24,Endeavor Air Inc.,ABY,ATL,False,False,1202,1157.0,0.0,-5.0,...,1210.0,1246.0,12.0,1304,-6.0,0.0,-1.0,1300-1359,1,0.0
2,2018-01-25,Endeavor Air Inc.,ABY,ATL,False,False,1202,1153.0,0.0,-9.0,...,1211.0,1251.0,11.0,1304,-2.0,0.0,-1.0,1300-1359,1,0.0
3,2018-01-26,Endeavor Air Inc.,ABY,ATL,False,False,1202,1150.0,0.0,-12.0,...,1207.0,1242.0,11.0,1304,-11.0,0.0,-1.0,1300-1359,1,0.0
4,2018-01-27,Endeavor Air Inc.,ABY,ATL,False,False,1400,1355.0,0.0,-5.0,...,1412.0,1448.0,11.0,1500,-1.0,0.0,-1.0,1500-1559,1,0.0


In [9]:
len(raw)

29193782

In [5]:
df_unique_origin = raw[['FlightDate', 'Origin']].drop_duplicates().rename(columns={'FlightDate': 'Date', 'Origin': 'Location'})
df_unique_dest = raw[['FlightDate', 'Dest']].drop_duplicates().rename(columns={'FlightDate': 'Date', 'Dest': 'Location'})

# Combine the two DataFrames
df_date_location = pd.concat([df_unique_origin, df_unique_dest]).drop_duplicates()

# Preview the result
print(len(df_date_location))
print(df_date_location)

581035
               Date Location
0        2018-01-23      ABY
1        2018-01-24      ABY
2        2018-01-25      ABY
3        2018-01-26      ABY
4        2018-01-27      ABY
...             ...      ...
26137438 2022-02-23      DRT
26854440 2022-05-17      SUN
28491644 2022-01-01      PIR
28591464 2022-01-01      PQI
28940111 2022-03-22      SUX

[581035 rows x 2 columns]


In [6]:
airports = df_date_location[['Location']].drop_duplicates()

In [7]:
# weather = pd.read_csv(r'C:\Users\ekait\Downloads\2018.csv')
# weather[weather['ASN00040289']=='ASN00040289']

In [44]:
airports

,Location
0,ABY
9,ATL
40,MOB
65,BUF
68,DFW
...,...
18807226,FOD
18807771,MCW
19081753,ILG
19724020,TBN


In [50]:
airport_locations = pd.read_csv(r'C:\Users\ekait\Downloads\airports.dat', header=None)

In [51]:
airport_locations

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7693,14106,Rogachyovo Air Base,Belaya,Russia,\N,ULDA,71.616699,52.478298,272,\N,\N,\N,airport,OurAirports
7694,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,\N,\N,\N,airport,OurAirports
7695,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,\N,\N,\N,airport,OurAirports
7696,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,\N,\N,airport,OurAirports


In [52]:
# airport_latlon = airport_locations[airport_locations[3]=='United States'][[1,2,3,4,6,7]].rename(columns={1:'Airport Name', 2:'City', 3:'Country', 4: 'Location', 6: 'Latitude', 7: 'Longitude'})
airport_latlon = airport_locations[[1,2,3,4,6,7]].rename(columns={1:'Airport Name', 2:'City', 3:'Country', 4: 'Location', 6: 'Latitude', 7: 'Longitude'})

In [57]:
airport_latlon_complete = pd.merge(airports, airport_latlon, on='Location', how='left').sort_values('Location').reset_index(drop=True)
airport_latlon_complete = airport_latlon_complete[['Location', 'Latitude', 'Longitude']]
print(airport_latlon_complete)

    Location   Latitude   Longitude
0        ABE  40.652100  -75.440804
1        ABI  32.411301  -99.681900
2        ABQ  35.040199 -106.609001
3        ABR  45.449100  -98.421799
4        ABY  31.535500  -84.194504
..       ...        ...         ...
383      XWA        NaN         NaN
384      YAK  59.503300 -139.660004
385      YKM  46.568199 -120.543999
386      YNG  41.260700  -80.679100
387      YUM  32.656601 -114.606003

[388 rows x 3 columns]


In [58]:
airport_latlon_complete.to_csv(r'C:\Users\ekait\Downloads\airport_list_2.csv')